In [ ]:
%pip install wikipedia wikipedia-api
%pip install langchain langchain-community langchain-huggingface langchain-utilites
%pip install faiss-cpu faiss-gpu
%pip install -U transformers
%pip install torch
%pip install bitsandbytes
%pip install accelerate

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11679 sha256=d169b1963e73705db3da5e2fff432ed3eafe1b140a340a2526bf6d3ee1fced93
  Stored in directory: /root/.cache/pip/wheels/5e/b6/c5/93f3dec388ae76edc830cb42901bb0232504dfc0df02fc50de
  Created wheel for wikipedia-api: filename=Wikipedia_API-0.7.1-py3-none-any.whl size=14346 sha256=e4c817b38add28c216da5ab42c985d5997da658a2ddfcdb9389fc46819c3f8ea
  Stored in directory: /root/.cache/pip/wheels/4c/96/18/b9201cc3e8b47b02b510460210cfd832ccf10c0c4dd0522962
Successfully built wikipedia wikipedia-api
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 44.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 67.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 407.7/407.7 kB 35.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import warnings
import wikipedia
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
import torch
from transformers import AutoProcessor, MllamaForConditionalGeneration, BitsAndBytesConfig
import requests
from PIL import Image
import re
from huggingface_hub import login
import io
import os

login(HF_TOKEN)

device = "cuda" if torch.cuda.is_available() else "cpu"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model_id = "meta-llama/Llama-3.2-11B-Vision-Instruct"
processor = AutoProcessor.from_pretrained(model_id)
model = MllamaForConditionalGeneration.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    quantization_config=bnb_config,
    device_map="auto",
)

embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2",
                                        encode_kwargs={'normalize_embeddings': True})

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/437 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/55.8k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/5.07k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
def get_wikipedia_pages(keywords, num_results=10):
    wikipedia.set_lang("en")
    titles = wikipedia.search(keywords, num_results)
    pages = []
    for t in titles:
        try:
            pages.append(wikipedia.page(t, auto_suggest=False))
        except Exception as e:
            warnings.warn(str(e))
    return pages

In [ ]:
def get_keywords(query, image):
    input_text = f"Your task is to write a few search keywords to find Wikipedia pages containing the relevant information to answer the question about the provided image. The keywords must be as specific as possible and must represent the information that is needed to answer the question in relation to the provided image. Don't write more than 3 keywords and special characters. Please print only keywords as output.\nQuestion: {query}\n"

    conversation = [
        {
          "role": "user",
          "content": [
              {"type": "text", "text": input_text},
              {"type": "image"},
            ],
        },
    ]
    prompt = processor.apply_chat_template(conversation, add_generation_prompt=True)

    inputs = processor(image, prompt, return_tensors="pt").to(device)

    outputs = model.generate(**inputs, max_new_tokens=150)
    full_output = processor.decode(outputs[0], skip_special_tokens=True)
    prompt_length = len(processor.decode(inputs["input_ids"][0], skip_special_tokens=True))

    answer = full_output[prompt_length:].strip()
    return answer

In [ ]:
def split_text(pages, chunk_size=1024, chunk_overlap=150):
    splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    chunks = []
    for page in pages:
        chunks.extend(splitter.split_text(page.content))
    return chunks

In [ ]:
def retrieve_relevant_texts(query, faiss_index, top_k=5):
    return faiss_index.similarity_search(query, k=top_k)

In [ ]:
def generate_answer_from_retrieved(retrieved_texts, query, image):
    context = "\n\n".join(retrieved_texts)
    input_text = f"Question: {query}\nYou can use the following information from Wikipedia to provide your answer.\n\nContext: {context}"

    conversation = [
        {
          "role": "user",
          "content": [
              {"type": "text", "text": input_text},
              {"type": "image"},
            ],
        },
    ]
    prompt = processor.apply_chat_template(conversation, add_generation_prompt=True)

    inputs = processor(image, prompt, return_tensors="pt").to(device)

    outputs = model.generate(**inputs, max_new_tokens=250)
    full_output = processor.decode(outputs[0], skip_special_tokens=True)
    prompt_length = len(processor.decode(inputs["input_ids"][0], skip_special_tokens=True))

    answer = full_output[prompt_length:].strip()
    return answer

In [ ]:
def generate_answer(query, image):

    keywords = get_keywords(query, image)

    pages = get_wikipedia_pages(keywords)

    text_chunks = split_text(pages)

    faiss_index = FAISS.from_texts(text_chunks, embedding_model)

    relevant_texts = retrieve_relevant_texts(query + "Keywords: " + keywords, faiss_index)
    relevant_texts = [text.page_content for text in relevant_texts]

    final_answer = generate_answer_from_retrieved(relevant_texts, query, image)

    return final_answer

In [ ]:
r = requests.get('https://upload.wikimedia.org/wikipedia/commons/thumb/5/50/Jacques-Louis_David_-_The_Emperor_Napoleon_in_His_Study_at_the_Tuileries_-_Google_Art_Project.jpg/220px-Jacques-Louis_David_-_The_Emperor_Napoleon_in_His_Study_at_the_Tuileries_-_Google_Art_Project.jpg', stream=True)
image = Image.open(io.BytesIO(r.content))

generate_answer("Who is the person in the picture?", image)

'The person in the picture is Napoleon Bonaparte.'

In [ ]:
%pip install duckduckgo-search

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 74.4 MB/s eta 0:00:00


In [ ]:
from langchain_community.tools import DuckDuckGoSearchRun

def generate_web_answer(query, image):
    keywords = get_keywords(query, image)

    search = DuckDuckGoSearchRun()
    search_results = search.run(keywords)

    final_answer = generate_answer_from_retrieved(search_results, query, image)

    return final_answer

In [ ]:
generate_web_answer("Who is the person in the picture?", image)

'The image is a portrait of Napoleon Bonaparte, a French military leader and emperor. The painting is a representation of Napoleon in his imperial attire, with a stern expression on his face. He is depicted standing in a room with a throne, a desk, and a clock on the wall. The painting is a realistic depiction of Napoleon, with attention to detail in his uniform and the surroundings.\n\n*Answer*: Napoleon Bonaparte.'